In [18]:
# se importa el conector de snowflake
import snowflake.connector 
# se importa del modulo de swnoflake la herramienta de pandas para ingestar un dataframe a una tabla de snowflake
from snowflake.connector.pandas_tools import write_pandas 
import pandas as pd
import wbgapi as wb
from sklearn import preprocessing
import streamlit as st
import snowflake.connector
import pickle

### Se crea la conexión con snowflake

In [13]:
#se crea conexión
conn = snowflake.connector.connect(
    user='grupods03',
    password='Henry2022#',
    account='nr28668.sa-east-1.aws')
    #warehouse='PRUEBA',
    #role ='ACCOUNTADMIN',
    #schema ='PUBLIC')

2022-10-25 11:21:50.350 INFO    snowflake.connector.connection: Snowflake Connector for Python Version: 2.8.0, Python Version: 3.10.5, Platform: Windows-10-10.0.19044-SP0
2022-10-25 11:21:50.352 INFO    snowflake.connector.connection: This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.


In [14]:
def execute_query(connection, query):
    cursor = connection.cursor() # se inicializa la conexión Creates a cursor object. Each statement will be executed in a new cursor object.
    cursor.execute(query)
    cursor.close()

### Se crea la database y el datawarehouse

In [ ]:
query = "Create database LAKE" # se crea database

execute_query(conn, query)

In [15]:
query = "Use database LAKE" # se inicializa database

execute_query(conn, query)

2022-10-25 11:22:17.589 INFO    snowflake.connector.cursor: query: [Use database LAKE]
2022-10-25 11:22:17.903 INFO    snowflake.connector.cursor: query execution done
2022-10-25 11:22:17.909 INFO    snowflake.connector.cursor: Number of results in first chunk: 1


In [ ]:
query = "Create warehouse DW_EV" # se crea un datawarehouse dentro de LAKE

execute_query(conn, query)

In [16]:
query = "Use warehouse DW_EV" # se inicializa datawarehouse

execute_query(conn, query)

2022-10-25 11:22:20.137 INFO    snowflake.connector.cursor: query: [Use warehouse DW_EV]
2022-10-25 11:22:20.418 INFO    snowflake.connector.cursor: query execution done
2022-10-25 11:22:20.419 INFO    snowflake.connector.cursor: Number of results in first chunk: 1


### Creamos las tablas y sus relaciones

In [194]:
'''Cuando queremos borrar una tabla
viewquery ="""
    DROP TABLE EV;
    """ # se borra la tabla INCOME

execute_query(conn, viewquery)
'''

In [302]:
viewquery ="""
    create or replace TABLE PAIS (
	CODIGO_PAIS VARCHAR(3) NOT NULL,
	ID_PAIS INT NOT NULL,
	NOMBRE VARCHAR(30) NOT NULL,
	constraint PKEY_1 primary key (ID_PAIS)
    );""" # se crea la tabla PAIS

execute_query(conn, viewquery)

2022-10-25 00:47:36.333 query: [create or replace TABLE PAIS ( CODIGO_PAIS VARCHAR(3) NOT NULL, ID_PAIS INT NOT ...]
2022-10-25 00:47:36.806 query execution done
2022-10-25 00:47:36.808 Number of results in first chunk: 1


In [303]:
viewquery ="""
    create or replace TABLE CONTINENTE (
	ID_CONTINENTE INT NOT NULL,
	CONTINENTE VARCHAR(20) NOT NULL,
	constraint PKEY_1 primary key (ID_CONTINENTE)
    );""" # se crea la tabla CONTINENTE

execute_query(conn, viewquery)

2022-10-25 00:47:39.743 query: [create or replace TABLE CONTINENTE ( ID_CONTINENTE INT NOT NULL, CONTINENTE VARC...]
2022-10-25 00:47:40.253 query execution done
2022-10-25 00:47:40.255 Number of results in first chunk: 1


In [304]:
viewquery ="""
    create or replace TABLE INDICADOR (
	ID_INDICADOR INT NOT NULL,
	CODIGO VARCHAR(20) NOT NULL,
	DESCRIPCION VARCHAR(16777216),
	constraint PKEY_1 primary key (ID_INDICADOR)
    );""" # se crea la tabla INDICADOR

execute_query(conn, viewquery)

2022-10-25 00:47:43.177 query: [create or replace TABLE INDICADOR ( ID_INDICADOR INT NOT NULL, CODIGO VARCHAR(20...]
2022-10-25 00:47:43.637 query execution done
2022-10-25 00:47:43.640 Number of results in first chunk: 1


In [305]:
viewquery ="""
    create or replace TABLE INCOME (
	CODIGO_INCOME VARCHAR(2) NOT NULL,
	ID_INCOME INT NOT NULL,
	INCOME VARCHAR(30),
	constraint PKEY_1 primary key (ID_INCOME)
    );""" # se crea la tabla INCOME

execute_query(conn, viewquery)

2022-10-25 00:47:48.634 query: [create or replace TABLE INCOME ( CODIGO_INCOME VARCHAR(2) NOT NULL, ID_INCOME IN...]
2022-10-25 00:47:49.111 query execution done
2022-10-25 00:47:49.113 Number of results in first chunk: 1


In [306]:
viewquery ="""
    create or replace TABLE EV (
	ID_PAIS INT NOT NULL,
	ID_CONTINENTE INT NOT NULL,
	ID_INDICADOR INT  NOT NULL,
	ID_INCOME INT NOT NULL,
	ANIO INTEGER NOT NULL,
	VALOR FLOAT,
	constraint FKEY_1 foreign key (ID_PAIS) references PAIS(ID_PAIS),
	constraint FKEY_2 foreign key (ID_CONTINENTE) references CONTINENTE(ID_CONTINENTE),
	constraint FKEY_3 foreign key (ID_INDICADOR) references INDICADOR(ID_INDICADOR),
	constraint FKEY_4 foreign key (ID_INCOME) references INCOME(ID_INCOME)
    );""" # se crea la tabla de hechos EV

execute_query(conn, viewquery)

2022-10-25 00:47:53.138 query: [create or replace TABLE EV ( ID_PAIS INT NOT NULL, ID_CONTINENTE INT NOT NULL, I...]
2022-10-25 00:47:53.759 query execution done
2022-10-25 00:47:53.761 Number of results in first chunk: 1


### Cargamos datos a las tablas dimensionales

Cargamos datos en la tabla continente

In [307]:
Continentes=['America','Europa','Asia','Africa','Oceania']
df=pd.DataFrame({"CONTINENTE":Continentes})

#Se codifica ID_PAIS a partir de CODIGO_PAIS
le_conti = LabelEncoder()
le_conti.fit(df['CONTINENTE'])
df['ID_CONTINENTE'] = le_conti.transform(df['CONTINENTE'])

#Se guarda la codificación
filehandler = open("le_conti.obj","wb")
pickle.dump(le_conti,filehandler)
filehandler.close()
 
write_pandas(conn, df, 'CONTINENTE') # se ingesta el dataframe pd a la tabla Continente


2022-10-25 00:47:58.406 query: [create temporary stage /* Python:snowflake.connector.pandas_tools.write_pandas()...]
2022-10-25 00:47:58.790 query execution done
2022-10-25 00:47:58.792 Number of results in first chunk: 1
2022-10-25 00:47:58.802 query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ 'file://C:\\Use...]
2022-10-25 00:47:59.182 query execution done
2022-10-25 00:48:01.769 query: [COPY INTO "CONTINENTE" /* Python:snowflake.connector.pandas_tools.write_pandas()...]
2022-10-25 00:48:03.276 query execution done
2022-10-25 00:48:03.278 Number of results in first chunk: 1


(True,
 1,
 5,
 [('fqgrm/file0.txt', 'LOADED', 5, 5, 1, 0, None, None, None, None)])

In [ ]:
#Esto es un ejemplo de como reutilizar el labelencoder
Continentesx2=['America','America','Europa','Europa','Asia','Asia','Africa','Africa','Oceania','Oceania']
dfx2=pd.DataFrame({"CONTINENTE":Continentesx2})

file = open("le_conti.obj",'rb')
le_loaded = pickle.load(file)
file.close()
 
dfx2['ID_Continentes'] = le_loaded.transform(dfx2['CONTINENTE'])

dfx2


Cargamos datos en la tabla PAIS

In [308]:
Nation = ['United States', 'Canada','Mexico','Costa Rica','Panama','Brazil','Argentina','Chile','Uruguay','Bolivia','Peru','Egypt, Arab Rep.','Libya','South Africa','Nigeria','Morocco','Australia','China','India','Thailand','Japan','Korea, Rep.','Israel','Saudi Arabia','Malaysia','Indonesia','Russian Federation','Turkiye','Spain','Bulgaria','France','Italy','Germany','United Kingdom','Norway','Sweden','Greece']
Nation_code= ['USA','CAN','MEX','CRI','PAN','BRA','ARG','CHL','URY','BOL','PER','EGY','LBY','ZAF','NGA','MAR','AUS','CHN','IND','THA','JPN','KOR','ISR','SAU','MYS','IDN','RUS','TUR','ESP','BGR','FRA','ITA','DEU','GBR','NOR','SWE','GRC']
df=pd.DataFrame({'CODIGO_PAIS':Nation_code,'NOMBRE': Nation}) #Creamos el dataframe para PAIS

#Se codifica ID_PAIS a partir de CODIGO_PAIS
le_nation = LabelEncoder()
le_nation.fit(df['CODIGO_PAIS'])
df['ID_PAIS'] = le_nation.transform(df['CODIGO_PAIS'])

#Se guarda la codificación
filehandler = open("le_nation.obj","wb")
pickle.dump(le_nation,filehandler)
filehandler.close()

write_pandas(conn, df, 'PAIS') # se ingesta el dataframe pd a la tabla pais



2022-10-25 00:48:09.811 query: [create temporary stage /* Python:snowflake.connector.pandas_tools.write_pandas()...]
2022-10-25 00:48:10.185 query execution done
2022-10-25 00:48:10.186 Number of results in first chunk: 1
2022-10-25 00:48:10.200 query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ 'file://C:\\Use...]
2022-10-25 00:48:10.583 query execution done
2022-10-25 00:48:13.024 query: [COPY INTO "PAIS" /* Python:snowflake.connector.pandas_tools.write_pandas() */ ("...]
2022-10-25 00:48:14.603 query execution done
2022-10-25 00:48:14.605 Number of results in first chunk: 1


(True,
 1,
 37,
 [('fnfyo/file0.txt', 'LOADED', 37, 37, 1, 0, None, None, None, None)])

Cargamos datos en la tabla INCOME

In [309]:
Income=['Alto ingreso','Ingreso medio alto','Ingreso medio bajo']
CODIGO_INC=['H','M','LM']
df = pd.DataFrame({'INCOME':Income, 'CODIGO_INCOME':CODIGO_INC}) #Se crea el dataframe para INCOME

#Se codifica ID_INCOME a partir de Codigo_income
le_income = LabelEncoder()
le_income.fit(df['CODIGO_INCOME'])
df['ID_INCOME'] = le_income.transform(df['CODIGO_INCOME'])

#Se guarda la codificación
filehandler = open("le_income.obj","wb")
pickle.dump(le_income,filehandler)
filehandler.close()

write_pandas(conn, df, 'INCOME') # se ingesta el dataframe pd a la tabla INCOME


2022-10-25 00:48:20.790 query: [create temporary stage /* Python:snowflake.connector.pandas_tools.write_pandas()...]
2022-10-25 00:48:21.184 query execution done
2022-10-25 00:48:21.186 Number of results in first chunk: 1
2022-10-25 00:48:21.195 query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ 'file://C:\\Use...]
2022-10-25 00:48:21.585 query execution done
2022-10-25 00:48:24.406 query: [COPY INTO "INCOME" /* Python:snowflake.connector.pandas_tools.write_pandas() */ ...]
2022-10-25 00:48:25.953 query execution done
2022-10-25 00:48:25.955 Number of results in first chunk: 1


(True,
 1,
 3,
 [('ftdyg/file0.txt', 'LOADED', 3, 3, 1, 0, None, None, None, None)])

Cargamos datos en la tabla INDICADOR

In [310]:
Indicador = ['Average precipitation in depth (mm per year)',
'emisiones de CO2 (kt)',
'crecimiento de la poblacion (% anual)',
'Educational attainment, at least completed primary, population 25+ years, female (%) (cumulative)',
'School enrollment, tertiary (% gross)',
'Tasa de alfabetizacion, total adultos (% de personas)',
'Hepatitis B (HepB3) immunization coverage among 1-year-olds (%)',
'Immunization, DPT (% of children ages 12-23 months)',
'Immunization, measles (% of children ages 12-23 months)',
'Incidence of HIV, all (per 1,000 uninfected population)',
'Mortality rate, under-5 (per 1,000 live births)',
'Number of deaths ages 5-9 years',
'Number of infant deaths (per 1,000 live births)',
'Polio (Pol3) immunization covergae among 1-year-olds (%)',
'prevalencia del consumo de tabaco, hombres',
'prevalencia del consumo de tabaco, mujeres',
'tasa de mortalidad materna (cada 100.000 nacidos vivos)',
'tasa de mortalidad menores de 5 años (por 1000nacidos vivos)',
'tasa de mortalidad, adultos hombres (por cada 1000 adultos)',
'tasa de mortalidad, adultos mujeres (por cada 1000 adultos)',
'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)',
'Access to clean fuels and technologies for cooking, rural (% of rural population)',
'Current education expenditure, primary (% of total expenditure in primary public institutions)',
'Current health expenditure (% of GDP)',
'desempleo total (% de la poblacion laboral)',
'esperanza de vida al nacer, hombres (años)',
'esperanza de vida al nacer, muejres (años)',
'esperanza de vida al nacer, Total (años)',
'gasto publico (% del pib)',
'gasto publico en educación, total (% del pbi)',
'GDP per capita (constant 2015 US$)',
'poblacion que vive en barrios marginales (% de la poblacion urbana)',
'Población Rural (% de la poblacion total)',
'poblacion urbana (%poblacion total)',
'Research and development expenditure (% of GDP)',
'Researchers in R&D (per million people)',
'tasa de recuento de la pobreza, multidimensional (%de la poblacion total)',
'Trade in services (% of GDP)'
]
Code_Indicador = ['AG.LND.PRCP.MM',
'EN.ATM.CO2E.KT',
'SP.POP.GROW',
'SE.PRM.CUAT.FE.ZS',
'SE.TER.ENRR',
'SE.ADT.LITR.ZS',
'CSV1 from WHO',
'SH.IMM.IDPT',
'SH.IMM.MEAS',
'SH.HIV.INCD.TL.P3',
'SH.DYN.MORT',
'SH.DTH.0509',
'SH.DTH.IMRT.IN',
'CSV2 from WHO',
'SH.PRV.SMOK.MA',
'SH.PRV.SMOK.FE',
'SH.STA.MMRT',
'SH.DYN.MORT',
'SP.DYN.AMRT.MA',
'SP.DYN.AMRT.FE',
'SH.ALC.PCAP.LI',
'EG.CFT.ACCS.RU.ZS',
'SE.XPD.CPRM.ZS',
'SH.XPD.CHEX.GD.ZS',
'SL.UEM.TOTL.NE.ZS',
'SP.DYN.LE00.MA.IN',
'SP.DYN.LE00.FE.IN',
'SP.DYN.LE00.IN',
'GC.XPN.TOTL.GD.ZS',
'SE.XPD.TOTL.GD.ZS',
'NY.GDP.PCAP.KD',
'EN.POP.SLUM.UR.ZS',
'SP.RUR.TOTL.ZS',
'SP.URB.TOTL.IN.ZS',
'GB.XPD.RSDV.GD.ZS',
'SP.POP.SCIE.RD.P6',
'SI.POV.MDIM',
'BG.GSR.NFSV.GD.ZS'
]
#Se crea el df para la tabla INDICADOR
df=pd.DataFrame({'CODIGO':Code_Indicador,'DESCRIPCION':Indicador}) 

#Se codifica ID_INDICADOR a partir de CODIGO
le_indicador = LabelEncoder()
le_indicador.fit(df['CODIGO'])
df['ID_INDICADOR'] = le_indicador.transform(df['CODIGO'])

#Se guarda la codificación
filehandler = open("le_indicador.obj","wb")
pickle.dump(le_indicador,filehandler)
filehandler.close()

write_pandas(conn, df, 'INDICADOR') # se ingesta el dataframe pd a la tabla indicador

2022-10-25 00:48:31.598 query: [create temporary stage /* Python:snowflake.connector.pandas_tools.write_pandas()...]
2022-10-25 00:48:31.959 query execution done
2022-10-25 00:48:31.961 Number of results in first chunk: 1
2022-10-25 00:48:31.975 query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ 'file://C:\\Use...]
2022-10-25 00:48:32.392 query execution done
2022-10-25 00:48:34.900 query: [COPY INTO "INDICADOR" /* Python:snowflake.connector.pandas_tools.write_pandas() ...]
2022-10-25 00:48:36.496 query execution done
2022-10-25 00:48:36.497 Number of results in first chunk: 1


(True,
 1,
 38,
 [('tnaml/file0.txt', 'LOADED', 38, 38, 1, 0, None, None, None, None)])

### Cargamos datos a la tabla de hechos

Ingestamos datos a la tabla de hechos sin los KPIs

In [29]:
Indicador_code_INCOMPLETE = ['AG.LND.PRCP.MM',
'EN.ATM.CO2E.KT',
'SP.POP.GROW',
'SE.PRM.CUAT.FE.ZS',
'SE.TER.ENRR',
'SE.ADT.LITR.ZS',
'CSV1 from WHO',
'SH.IMM.IDPT',
'SH.IMM.MEAS',
'SH.HIV.INCD.TL.P3',
'SH.DTH.0509',
'SH.DTH.IMRT.IN',
'CSV24 from WHO',
'SH.PRV.SMOK.MA',
'SH.PRV.SMOK.FE',
'SH.DYN.MORT',
'SP.DYN.AMRT.MA',
'SP.DYN.AMRT.FE',
'SH.ALC.PCAP.LI',
'EG.CFT.ACCS.RU.ZS',
'SE.XPD.CPRM.ZS',
'SH.XPD.CHEX.GD.ZS',
'SL.UEM.TOTL.NE.ZS',
'SP.DYN.LE00.MA.IN',
'SP.DYN.LE00.FE.IN',
'SE.XPD.TOTL.GD.ZS',
'EN.POP.SLUM.UR.ZS',
'SP.URB.TOTL.IN.ZS',
'GB.XPD.RSDV.GD.ZS',
'SP.POP.SCIE.RD.P6',
'SI.POV.MDIM',
'BG.GSR.NFSV.GD.ZS'
]
df2=wb.data.DataFrame(Indicador_code_INCOMPLETE, 
                  Nation_code, 
                  mrv= 30,
                  columns='series',
                  numericTimeKeys=True
                  )
df2=df2.reset_index()
df2=df2.melt(id_vars=['economy','time'], var_name="ID_INDICADOR", value_name="VALOR")

Ingesta a tabla de hechos los indicadores de KPIs diferenciando desde el año que existen datos

In [5]:
Nation = ['United States', 'Canada','Mexico','Costa Rica','Panama','Brazil','Argentina','Chile','Uruguay','Bolivia','Peru','Egypt, Arab Rep.','Libya','South Africa','Nigeria','Morocco','Australia','China','India','Thailand','Japan','Korea, Rep.','Israel','Saudi Arabia','Malaysia','Indonesia','Russian Federation','Turkiye','Spain','Bulgaria','France','Italy','Germany','United Kingdom','Norway','Sweden','Greece']
Nation_code= ['USA','CAN','MEX','CRI','PAN','BRA','ARG','CHL','URY','BOL','PER','EGY','LBY','ZAF','NGA','MAR','AUS','CHN','IND','THA','JPN','KOR','ISR','SAU','MYS','IDN','RUS','TUR','ESP','BGR','FRA','ITA','DEU','GBR','NOR','SWE','GRC']


In [33]:
#Población rural desde 1960
PR62=wb.data.DataFrame('SP.RUR.TOTL.ZS', 
                  Nation_code, 
                  mrv= 62,
                  columns='series',
                  numericTimeKeys=True
                  )
PR62=PR62.reset_index()

In [34]:
#Esperanza de vida desde 1960
EV61=wb.data.DataFrame('SP.DYN.LE00.IN', 
                  Nation_code, 
                  mrv= 61,
                  columns='series',
                  numericTimeKeys=True
                  )
EV61=EV61.reset_index()
leftmerge=pd.merge(PR62,EV61, how="left", on=['economy', 'time'] )


In [35]:
#Ingreso per capita desde 1960
IPC62=wb.data.DataFrame('NY.GDP.PCAP.KD', 
                  Nation_code, 
                  mrv= 62,
                  columns='series',
                  numericTimeKeys=True
                  )
IPC62=IPC62.reset_index()
leftmerge=pd.merge(EV61,IPC62, how="left", on=['economy', 'time'] )

In [36]:
#Mortalidad materna desde 2000
MM18=wb.data.DataFrame('SH.STA.MMRT', 
                  Nation_code, 
                  mrv= 18,
                  columns='series',
                  numericTimeKeys=True
                  )
MM18=MM18.reset_index()
leftmerge=pd.merge(leftmerge,MM18, how="left", on=['economy', 'time'] )

In [44]:
#Inversion publica desde 1990
GS31=wb.data.DataFrame('GC.XPN.TOTL.GD.ZS', 
                  Nation_code, 
                  mrv= 31,
                  columns='series',
                  numericTimeKeys=True
                  )
GS31=GS31.reset_index()
leftmerge=pd.merge(leftmerge,GS31, how="left", on=['economy', 'time'] )

In [38]:
#GMortalidad infantil desde 1969
MC52=wb.data.DataFrame('SH.DYN.MORT', 
                  Nation_code, 
                  mrv= 52,
                  columns='series',
                  numericTimeKeys=True
                  )
MC52=MC52.reset_index()
leftmerge=pd.merge(leftmerge,MC52, how="left", on=['economy', 'time'] )

In [39]:
#Hacemos el melt de los datos para poder concatenar luego a la tabla de hechos
leftmerge=leftmerge.melt(id_vars=['economy','time'], var_name="ID_INDICADOR", value_name="VALOR")

In [40]:
leftmerge

,economy,time,ID_INDICADOR,VALOR
0,ARG,1960,SP.DYN.LE00.IN,65.055
1,ARG,1961,SP.DYN.LE00.IN,65.176
2,ARG,1962,SP.DYN.LE00.IN,65.269
3,ARG,1963,SP.DYN.LE00.IN,65.348
4,ARG,1964,SP.DYN.LE00.IN,65.426
...,...,...,...,...
11280,ZAF,2016,SH.DYN.MORT,35.200
11281,ZAF,2017,SH.DYN.MORT,34.600
11282,ZAF,2018,SH.DYN.MORT,33.900
11283,ZAF,2019,SH.DYN.MORT,33.000


In [318]:
#Agregamos los indicadores de KPI's a la tabla de hechos
hechos=pd.concat([df2,leftmerge])
#Renombramos las columnas economy y time
hechos=hechos.rename(columns={'economy':'CODIGO_PAIS','time':'ANIO'})

In [319]:
#Cargamos codificación guardada anteriormente para indicador
file = open("le_indicador.obj",'rb')
le_loaded = pickle.load(file)
file.close()
#transformamos ID_INDICADOR
hechos['ID_INDICADOR'] = le_loaded.transform(hechos['ID_INDICADOR'])

Creamos la columna Id_continente a partir de los países

In [320]:
lista_america=['ARG','CAN','MEX','USA','PAN','BRA','ARG','CHL','URY','BOL','PER','CRI']
lista_africa=['EGY','LBY','ZAF','NGA','MAR']
lista_asia=['CHN','IND','THA','JPN','KOR','ISR','SAU','MYS','IDN','RUS','TUR']
lista_europa=['ESP','BGR','FRA','ITA','DEU','GBR','NOR','SWE','GRC']
lista_oceania=['AUS']

def categoria_pais(row):
    if row['CODIGO_PAIS'] in lista_america:
        return "America"
    if row['CODIGO_PAIS'] in lista_europa:
        return "Europa"
    if row['CODIGO_PAIS'] in lista_asia:
        return "Asia"
    if row['CODIGO_PAIS'] in lista_africa:
        return "Africa"
    if row['CODIGO_PAIS'] in lista_oceania:
        return "Oceania"

hechos['CONTINENTE'] = hechos.apply(lambda row: categoria_pais(row), axis=1)

#Abrimos encoder guardado
file = open("le_conti.obj",'rb')
le_loaded = pickle.load(file)
file.close()

#Codificamos columna CONTINENTE como ID_CONTINENTE
hechos['ID_CONTINENTE'] = le_loaded.transform(hechos['CONTINENTE'])

#label_encoder = preprocessing.LabelEncoder()
#numero_indicador = label_encoder.fit_transform(hechos['CONTINENTE'])
#hechos['ID_CONTINENTE'] = hechos.apply(lambda row: numero_indicador(row), axis=1)
#hechos['ID_CONTINENTE'] = hechos.apply(lambda row: categoria_pais(row), axis=1)

Creamos la columna ID_INGRESO a partir de una lista y del nombre del pais

In [321]:
lista_alto=['CAN','USA','PAN','CHL','URY','AUS','JPN','KOR','ISR','SAU','ESP','FRA','ITA','DEU','GBR','NOR','SWE','GRC']
lista_medioalto=['ARG','MEX','BRA','ARG','CRI','PER','BGR','TUR','LBY','ZAF','RUS']
lista_mediobajo=['CHN','THA','MYS','BOL','EGY','NGA','MAR','IND','IDN']


def ingreso_pais(row):
    if row['CODIGO_PAIS'] in lista_alto:
        return 'H'
    if row['CODIGO_PAIS'] in lista_medioalto:
        return 'M'
    if row['CODIGO_PAIS'] in lista_mediobajo:
        return 'LM'
    
hechos['CODIGO_INCOME'] = hechos.apply(lambda row: ingreso_pais(row), axis=1)

#Abrimos encoder guardado
file = open("le_income.obj",'rb')
le_loaded = pickle.load(file)
file.close()

#Codificamos columna CONTINENTE como ID_CONTINENTE
hechos['ID_INCOME'] = le_loaded.transform(hechos['CODIGO_INCOME'])



Codificamos la columna ID_PAIS

In [322]:
#Abrimos encoder guardado
file = open("le_nation.obj",'rb')
le_loaded = pickle.load(file)
file.close()

#Codificamos columna CODIGO_PAIS como ID_PAIS
hechos['ID_PAIS'] = le_loaded.transform(hechos['CODIGO_PAIS'])

In [323]:
hechos=hechos.drop(columns=['CODIGO_PAIS','CONTINENTE', 'CODIGO_INCOME'])

In [324]:
hechos

,ANIO,ID_INDICADOR,VALOR,ID_CONTINENTE,ID_INCOME,ID_PAIS
0,1992,0,591.0,1,2,0
1,1993,0,NaN,1,2,0
2,1994,0,NaN,1,2,0
3,1995,0,NaN,1,2,0
4,1996,0,NaN,1,2,0
...,...,...,...,...,...,...
11280,2016,18,35.2,0,2,36
11281,2017,18,34.6,0,2,36
11282,2018,18,33.9,0,2,36
11283,2019,18,33.0,0,2,36


Cargamos lo indicadores "Hepatitis B", "Gasto en salud" y "Polio" desde un CSV descargado de la página de WHO

In [211]:
Hepa=pd.read_csv('https://raw.githubusercontent.com/grupohenryds03/esperanza_vida/main/datasets/HepatitisB.csv')

In [325]:
Countries = ['United States', 'Canada','Mexico','Costa Rica','Panama','Brazil','Argentina','Chile','Uruguay','Bolivia','Peru','Egypt, Arab Rep.','Libya','South Africa','Nigeria','Morocco','Australia','China','India','Thailand','Japan','Korea, Rep.','Israel','Saudi Arabia','Malaysia','Indonesia','Russian Federation','Turkiye','Spain','Bulgaria','France','Italy','Germany','United Kingdom','Norway','Sweden','Greece']
Nation_code= ['USA','CAN','MEX','CRI','PAN','BRA','ARG','CHL','URY','BOL','PER','EGY','LBY','ZAF','NGA','MAR','AUS','CHN','IND','THA','JPN','KOR','ISR','SAU','MYS','IDN','RUS','TUR','ESP','BGR','FRA','ITA','DEU','GBR','NOR','SWE','GRC']

Hepatitis=pd.DataFrame({2000:[],2001:[],2002:[],2003:[]})
for n in Nation_code:
    Hepa2=Hepa[['Location','Period', 'Value','SpatialDimValueCode']]
    Hepa2=Hepa2.loc[Hepa2['SpatialDimValueCode']==n]
    Hepa2=Hepa2.sort_values(by=['Period'])
    Hepa2=Hepa2.set_index('Period').T
    Hepa2=Hepa2.drop(['Location','SpatialDimValueCode'])
    Hepa2=Hepa2.rename(index={'Value':n})
    Hepatitis=pd.concat([Hepatitis,Hepa2])

Hepatitis=Hepatitis.reset_index()
Hepatitis=Hepatitis.melt(id_vars=['index'], var_name="ANIO", value_name="VALOR")
Hepatitis['ID_INDICADOR']=2
Hepatitis=Hepatitis.rename(columns={"index":"CODIGO_PAIS"})

In [215]:
Polio=pd.read_csv('https://raw.githubusercontent.com/grupohenryds03/esperanza_vida/main/datasets/Polio.csv')

In [326]:
Poliodf=pd.DataFrame({2000:[],2001:[],2002:[],2003:[]})
for n in Nation_code:
    Polio2=Polio[['Location','Period', 'Value', 'SpatialDimValueCode']]
    Polio2=Polio2.loc[Polio2['SpatialDimValueCode']==n]
    Polio2=Polio2.sort_values(by=['Period'])
    Polio2=Polio2.set_index('Period').T
    Polio2=Polio2.drop(['Location', 'SpatialDimValueCode' ])
    Polio2=Polio2.rename(index={'Value':n})
    Poliodf=pd.concat([Poliodf,Polio2])
    
Poliodf=Poliodf.reset_index()
Poliodf=Poliodf.melt(id_vars=['index'], var_name="ANIO", value_name="VALOR")
Poliodf['ID_INDICADOR']=3
Poliodf=Poliodf.rename(columns={"index":"CODIGO_PAIS"})

Concatenamos las tablas del WHO

In [327]:
WHO=pd.concat([Hepatitis,Poliodf])

Agregamos las columnas ID_CONTINENTE, ID_INCOME y ID_PAIS con el mismo criterio de la tabla Hechos

In [328]:
lista_america=['ARG','CAN','MEX','USA','PAN','BRA','ARG','CHL','URY','BOL','PER','CRI']
lista_africa=['EGY','LBY','ZAF','NGA','MAR']
lista_asia=['CHN','IND','THA','JPN','KOR','ISR','SAU','MYS','IDN','RUS','TUR']
lista_europa=['ESP','BGR','FRA','ITA','DEU','GBR','NOR','SWE','GRC']
lista_oceania=['AUS']

def categoria_pais(row):
    if row['CODIGO_PAIS'] in lista_america:
        return "America"
    if row['CODIGO_PAIS'] in lista_europa:
        return "Europa"
    if row['CODIGO_PAIS'] in lista_asia:
        return "Asia"
    if row['CODIGO_PAIS'] in lista_africa:
        return "Africa"
    if row['CODIGO_PAIS'] in lista_oceania:
        return "Oceania"

WHO['CONTINENTE'] = WHO.apply(lambda row: categoria_pais(row), axis=1)

#Abrimos encoder guardado
file = open("le_conti.obj",'rb')
le_loaded = pickle.load(file)
file.close()

#Codificamos columna CONTINENTE como ID_CONTINENTE
WHO['ID_CONTINENTE'] = le_loaded.transform(WHO['CONTINENTE'])


lista_alto=['CAN','USA','PAN','CHL','URY','AUS','JPN','KOR','ISR','SAU','ESP','FRA','ITA','DEU','GBR','NOR','SWE','GRC']
lista_medioalto=['ARG','MEX','BRA','ARG','CRI','PER','BGR','TUR','LBY','ZAF','RUS']
lista_mediobajo=['CHN','THA','MYS','BOL','EGY','NGA','MAR','IND','IDN']

def ingreso_pais(row):
    if row['CODIGO_PAIS'] in lista_alto:
        return 'H'
    if row['CODIGO_PAIS'] in lista_medioalto:
        return 'M'
    if row['CODIGO_PAIS'] in lista_mediobajo:
        return 'LM'
    
WHO['CODIGO_INCOME'] = WHO.apply(lambda row: ingreso_pais(row), axis=1)

#Abrimos encoder guardado
file = open("le_income.obj",'rb')
le_loaded = pickle.load(file)
file.close()

#Codificamos columna INCOME como ID_INCOME
WHO['ID_INCOME'] = le_loaded.transform(WHO['CODIGO_INCOME'])

#Abrimos encoder guardado
file = open("le_nation.obj",'rb')
le_loaded = pickle.load(file)
file.close()

#Codificamos columna CODIGO_PAIS como ID_PAIS
WHO['ID_PAIS'] = le_loaded.transform(WHO['CODIGO_PAIS'])


In [329]:
WHO=WHO.drop(columns=['CODIGO_PAIS','CONTINENTE','CODIGO_INCOME'])

In [330]:
WHO.head()

,ANIO,VALOR,ID_INDICADOR,ID_CONTINENTE,ID_INCOME,ID_PAIS
0,2000,90,2,1,0,35
1,2000,NaN,2,1,0,5
2,2000,97,2,1,2,23
3,2000,89,2,1,2,8
4,2000,NaN,2,1,0,27


Concatenamos las tabla WHO con la tabla de hechos

In [331]:
hechos=pd.concat([hechos,WHO])

In [332]:
hechos

,ANIO,ID_INDICADOR,VALOR,ID_CONTINENTE,ID_INCOME,ID_PAIS
0,1992,0,591.0,1,2,0
1,1993,0,NaN,1,2,0
2,1994,0,NaN,1,2,0
3,1995,0,NaN,1,2,0
4,1996,0,NaN,1,2,0
...,...,...,...,...,...,...
809,2021,3,91,3,0,9
810,2021,3,93,3,0,13
811,2021,3,97,3,0,26
812,2021,3,98,3,0,31


Cargamos los datos de la tabla hechos a la tabla EV de snowflake

In [333]:
write_pandas(conn, hechos, 'EV') # se ingesta el dataframe pd a la tabla EV
# se cortan las conexiones
conn.close()

2022-10-25 00:53:40.729 query: [create temporary stage /* Python:snowflake.connector.pandas_tools.write_pandas()...]
2022-10-25 00:53:42.499 query execution done
2022-10-25 00:53:42.500 Number of results in first chunk: 1
2022-10-25 00:53:42.665 query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ 'file://C:\\Use...]
2022-10-25 00:53:43.048 query execution done
2022-10-25 00:53:46.283 query: [COPY INTO "EV" /* Python:snowflake.connector.pandas_tools.write_pandas() */ ("AN...]
2022-10-25 00:53:48.133 query execution done
2022-10-25 00:53:48.135 Number of results in first chunk: 1
2022-10-25 00:53:48.137 closed
2022-10-25 00:53:48.364 No async queries seem to be running, deleting session


In [211]:
#hechos.to_csv('hechos_1.csv', index=False)

### Hacemos consultas

In [3]:
#se crea conexión
cnn = snowflake.connector.connect(
    user='grupods03',
    password='Henry2022#',
    account='nr28668.sa-east-1.aws',
    warehouse='DW_EV',
    database="LAKE")
    #role ='ACCOUNTADMIN',
    #schema ='PUBLIC')

2022-10-25 11:03:44.365 INFO    snowflake.connector.connection: Snowflake Connector for Python Version: 2.8.0, Python Version: 3.10.5, Platform: Windows-10-10.0.19044-SP0
2022-10-25 11:03:44.367 INFO    snowflake.connector.connection: This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
2022-10-25 11:03:44.371 INFO    snowflake.connector.connection: Setting use_openssl_only mode to False


In [ ]:
#se crea conexión
cnn1 = snowflake.connector.connect(
    user=st.secrets.snowflake.user,
    password=st.secrets.snowflake.password,
    account=st.secrets.snowflake.account,
    warehouse=st.secrets.snowflake.warehouse,
    database=st.secrets.snowflake.database)
    #role ='ACCOUNTADMIN',
    #schema ='PUBLIC')

In [17]:
sql="SELECT * FROM PAIS"
pais=pd.read_sql(sql,cnn)
pais.head()

c:\Users\Jhovany Lara\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
2022-10-24 12:41:14.134 query: [SELECT * FROM PAIS]
2022-10-24 12:41:16.869 query execution done
2022-10-24 12:41:16.873 Number of results in first chunk: 37


,ID_PAIS,CODIGO_PAIS,NOMBRE
0,0,ARG,Argentina
1,1,AUS,Australia
2,2,BGR,Bulgaria
3,3,BOL,Bolivia
4,4,BRA,Brazil


In [38]:
Dicc_id_pais=pd.Series(pais.ID_PAIS.values,index=pais.CODIGO_PAIS).to_dict()
Dicc_id_pais

{'ARG': 0,
 'AUS': 1,
 'BGR': 4,
 'BOL': 2,
 'BRA': 3,
 'CAN': 5,
 'CHL': 6,
 'CHN': 7,
 'CRI': 8,
 'DEU': 11,
 'EGY': 9,
 'ESP': 30,
 'FRA': 10,
 'GBR': 34,
 'GRC': 12,
 'IDN': 14,
 'IND': 13,
 'ISR': 15,
 'ITA': 16,
 'JPN': 17,
 'KOR': 18,
 'LBY': 19,
 'MAR': 22,
 'MEX': 21,
 'MYS': 20,
 'NGA': 23,
 'NOR': 24,
 'PAN': 25,
 'PER': 26,
 'RUS': 27,
 'SAU': 28,
 'SWE': 31,
 'THA': 32,
 'TUR': 33,
 'URY': 36,
 'USA': 35,
 'ZAF': 29}

In [ ]:
Dicc_pais=pd.Series(pais.NOMBRE.values,index=pais.ID_PAIS).to_dict()
Dicc_pais

In [6]:
pais.to_csv('Incomplete.csv')

In [23]:
sql="SELECT * FROM INDICADOR "
indic=pd.read_sql(sql,cnn)
indic.head()

c:\Users\Jhovany Lara\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
2022-10-24 12:58:57.761 query: [SELECT * FROM INDICADOR]
2022-10-24 12:59:00.476 query execution done
2022-10-24 12:59:00.478 Number of results in first chunk: 38


,ID_INDICADOR,CODIGO,DESCRIPCION
0,1,AG.LND.PRCP.MM,Average precipitation in depth (mm per year)
1,2,EN.ATM.CO2E.KT,emisiones de CO2 (kt)
2,3,SP.POP.GROW,crecimiento de la poblacion (% anual)
3,4,SE.PRM.CUAT.FE.ZS,"Educational attainment, at least completed pri..."
4,5,SE.TER.ENRR,"School enrollment, tertiary (% gross)"


In [24]:
label_encoder = preprocessing.LabelEncoder()
indic['ID_INDICADOR'] = label_encoder.fit_transform(indic['CODIGO'])
indic.head()

,ID_INDICADOR,CODIGO,DESCRIPCION
0,0,AG.LND.PRCP.MM,Average precipitation in depth (mm per year)
1,4,EN.ATM.CO2E.KT,emisiones de CO2 (kt)
2,32,SP.POP.GROW,crecimiento de la poblacion (% anual)
3,10,SE.PRM.CUAT.FE.ZS,"Educational attainment, at least completed pri..."
4,11,SE.TER.ENRR,"School enrollment, tertiary (% gross)"


In [39]:
Dic_indicador=pd.Series(indic.CODIGO.values,index=indic.ID_INDICADOR).to_dict()
Dic_indicador

{0: 'AG.LND.PRCP.MM',
 4: 'EN.ATM.CO2E.KT',
 32: 'SP.POP.GROW',
 10: 'SE.PRM.CUAT.FE.ZS',
 11: 'SE.TER.ENRR',
 9: 'SE.ADT.LITR.ZS',
 2: 'CSV from WHO',
 19: 'SH.IMM.IDPT',
 20: 'SH.IMM.MEAS',
 18: 'SH.HIV.INCD.TL.P3',
 17: 'SH.DYN.MORT',
 15: 'SH.DTH.0509',
 16: 'SH.DTH.IMRT.IN',
 22: 'SH.PRV.SMOK.MA',
 21: 'SH.PRV.SMOK.FE',
 23: 'SH.STA.MMRT',
 28: 'SP.DYN.AMRT.MA',
 27: 'SP.DYN.AMRT.FE',
 14: 'SH.ALC.PCAP.LI',
 3: 'EG.CFT.ACCS.RU.ZS',
 12: 'SE.XPD.CPRM.ZS',
 24: 'SH.XPD.CHEX.GD.ZS',
 26: 'SL.UEM.TOTL.NE.ZS',
 31: 'SP.DYN.LE00.MA.IN',
 29: 'SP.DYN.LE00.FE.IN',
 30: 'SP.DYN.LE00.IN',
 7: 'GC.XPN.TOTL.GD.ZS',
 13: 'SE.XPD.TOTL.GD.ZS',
 8: 'NY.GDP.PCAP.KD',
 5: 'EN.POP.SLUM.UR.ZS',
 34: 'SP.RUR.TOTL.ZS',
 35: 'SP.URB.TOTL.IN.ZS',
 6: 'GB.XPD.RSDV.GD.ZS',
 33: 'SP.POP.SCIE.RD.P6',
 25: 'SI.POV.MDIM',
 1: 'BG.GSR.NFSV.GD.ZS'}

In [27]:
sql="SELECT * FROM INCOME "
income=pd.read_sql(sql,cnn)
income.head()

c:\Users\Jhovany Lara\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
2022-10-24 13:03:42.545 query: [SELECT * FROM INCOME]
2022-10-24 13:03:45.071 query execution done
2022-10-24 13:03:45.074 Number of results in first chunk: 3


,ID_INCOME,CODIGO_INCOME,INCOME
0,0,H,Alto ingreso
1,2,M,Ingreso medio alto
2,1,LM,Ingreso medio bajo


In [28]:
label_encoder = preprocessing.LabelEncoder()
income['ID_INCOME'] = label_encoder.fit_transform(income['CODIGO_INCOME'])
income.head()

,ID_INCOME,CODIGO_INCOME,INCOME
0,0,H,Alto ingreso
1,2,M,Ingreso medio alto
2,1,LM,Ingreso medio bajo


In [40]:
Dic_incom=pd.Series(income.INCOME.values,index=income.ID_INCOME).to_dict()
Dic_incom

{0: 'Alto ingreso', 2: 'Ingreso medio alto', 1: 'Ingreso medio bajo'}

In [31]:
sql="SELECT * FROM CONTINENTE "
contin=pd.read_sql(sql,cnn)
contin.head()

c:\Users\Jhovany Lara\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
2022-10-24 13:07:21.949 query: [SELECT * FROM CONTINENTE]
2022-10-24 13:07:24.450 query execution done
2022-10-24 13:07:24.452 Number of results in first chunk: 5


,ID_CONTINENTE,CONTINENTE
0,1,America
1,2,Europa
2,3,Asia
3,4,Africa
4,5,Oceania


In [32]:
label_encoder = preprocessing.LabelEncoder()
contin['ID_CONTINENTE'] = label_encoder.fit_transform(contin['CONTINENTE'])
contin.head()

,ID_CONTINENTE,CONTINENTE
0,1,America
1,3,Europa
2,2,Asia
3,0,Africa
4,4,Oceania


In [41]:
Dic_contin=pd.Series(contin.CONTINENTE.values,index=contin.ID_CONTINENTE).to_dict()
Dic_contin

{1: 'America', 3: 'Europa', 2: 'Asia', 0: 'Africa', 4: 'Oceania'}

In [44]:
dfcont = pd.DataFrame(Dic_contin.items(), columns=['ID_CONTINENTE', 'CONTINENTE']) # se crea dataframe con pandas
dfcont

,ID_CONTINENTE,CONTINENTE
0,1,America
1,3,Europa
2,2,Asia
3,0,Africa
4,4,Oceania


In [ ]:
from sklearn.preprocessing import OneHotEncoder
paiscode=OneHotEncoder()


In [7]:
sql="SELECT * FROM EV"
ALL2=pd.read_sql(sql,cnn)
ALL2.head()

c:\Users\Jhovany Lara\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,ID_PAIS,ANIO,ID_INDICADOR,VALOR,ID_CONTINENTE,ID_INCOME
0,0,1992,38,3.71,1,2
1,0,1993,38,3.96,1,2
2,0,1994,38,4.06,1,2
3,0,1995,38,4.28,1,2
4,0,1996,38,4.53,1,2


In [8]:
ALL2.to_csv('Complete.csv')

In [ ]:
sql ="""SELECT p.CODIGO_PAIS, e.ANIO, e.VALOR, i.DESCRIPCION as INDICADOR 
        FROM EV e 
        JOIN INDICADOR i 
        ON (e.ID_INDICADOR=i.ID_INDICADOR)
        JOIN PAIS p
        on (e.ID_PAIS=p.ID_PAIS)
        WHERE e.ID_INDICADOR=35 AND e.ANIO>1960 AND e.ANIO<=2020  """
probando=pd.read_sql(sql,cnn)
#AND e.ANIO>1960 AND e.ANIO<=2020

In [18]:
probando

,CODIGO_PAIS,ANIO,VALOR,INDICADOR
0,ARG,1992,72.03,"esperanza de vida al nacer, Total (años)"
1,ARG,1993,72.25,"esperanza de vida al nacer, Total (años)"
2,ARG,1994,72.45,"esperanza de vida al nacer, Total (años)"
3,ARG,1995,72.65,"esperanza de vida al nacer, Total (años)"
4,ARG,1996,72.84,"esperanza de vida al nacer, Total (años)"
...,...,...,...,...
1068,ZAF,2016,63.15,"esperanza de vida al nacer, Total (años)"
1069,ZAF,2017,63.54,"esperanza de vida al nacer, Total (años)"
1070,ZAF,2018,63.86,"esperanza de vida al nacer, Total (años)"
1071,ZAF,2019,64.13,"esperanza de vida al nacer, Total (años)"


In [11]:
#Consulta todos los países con indicador 2 --> Se hizo doble JOIN <--
sql ="""SELECT e.ID_PAIS, e.ANIO, e.VALOR, i.DESCRIPCION as INDICADOR 
    FROM EV e 
    JOIN INDICADOR i 
    ON (e.ID_INDICADOR=i.ID_INDICADOR) 
    WHERE e.ID_INDICADOR=28 AND e.ANIO>1960 AND e.ANIO<=2020 AND e.ID_PAIS=0""" 
arg=pd.read_sql(sql,cnn)

c:\Users\Jhovany Lara\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [ ]:
arg

In [ ]:
w=arg['VALOR'].tolist()
w

In [53]:
df=df.pivot(index=['ID_PAIS', 'INDICADOR'], columns='ANIO')

In [ ]:
arg2

In [57]:
#Consulta todos los países con indicador 2 --> Se hizo doble JOIN <--
sql ="SELECT p.NOMBRE, e.ANIO, e.VALOR, i.CODIGO as INDICADOR FROM EV e JOIN PAIS p ON (e.ID_PAIS=p.ID_PAIS) JOIN INDICADOR i ON (e.ID_INDICADOR=i.ID_INDICADOR) WHERE e.ID_INDICADOR=2" 
Indicador2=pd.read_sql(sql,cnn)

c:\Users\Jhovany Lara\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [58]:
Indicador2

,NOMBRE,ANIO,VALOR,INDICADOR
0,Argentina,1992,107930.0000,EN.ATM.CO2E.KT
1,Argentina,1992,107930.0000,EN.ATM.CO2E.KT
2,Argentina,1993,110260.0000,EN.ATM.CO2E.KT
3,Argentina,1993,110260.0000,EN.ATM.CO2E.KT
4,Argentina,1994,111910.0000,EN.ATM.CO2E.KT
...,...,...,...,...
2215,South Africa,2019,439640.0146,EN.ATM.CO2E.KT
2216,South Africa,2020,219820.6440,EN.ATM.CO2E.KT
2217,South Africa,2020,219820.6440,EN.ATM.CO2E.KT
2218,South Africa,2021,1.2473,EN.ATM.CO2E.KT


In [ ]:
mun2=Indicador2.pivot(index=['NOMBRE', 'INDICADOR'], columns='ANIO')
mun2

In [31]:
#Consulta un país (ARG) con indicador 2 en letra --> Se hizo doble JOIN <--
sql ="SELECT p.NOMBRE, e.ANIO, e.VALOR, i.DESCRIPCION FROM EV e JOIN PAIS p ON (e.ID_PAIS=p.ID_PAIS) JOIN INDICADOR i ON (e.ID_INDICADOR=i.ID_INDICADOR) WHERE e.ID_INDICADOR=2 AND e.ID_PAIS='ARG'" 
Indicador2ARG_LETRA=pd.read_sql(sql,cnn)

c:\Users\Jhovany Lara\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [ ]:
Indicador2ARG_LETRA

In [36]:
cnn.close

<bound method SnowflakeConnection.close of <snowflake.connector.connection.SnowflakeConnection object at 0x00000255505865F0>>